In [1]:
import sys
import os
import numpy as np
import pandas as pd

In [2]:
pkg_root = os.path.dirname(os.path.abspath(os.path.curdir))
sys.path.append(pkg_root)

In [3]:
from extb.gff import parse
from extb.utils import magic_open
from extb.GenomicAnnotation import GenomicAnnotation

In [4]:
planaria_example_gff = """v31.000002      WormBase_imported       mRNA    419767  423636  .       +       .       ID=transcript:mk4.000002.13;Parent=gene:mk4.000002.13;Name=mk4
v31.000002      WormBase_imported       exon    419767  420048  .       +       .       ID=exon:mk4.000002.13.1;Parent=transcript:mk4.000002.13
v31.000002      WormBase_imported       exon    420095  420198  .       +       .       ID=exon:mk4.000002.13.2;Parent=transcript:mk4.000002.13
v31.000002      WormBase_imported       exon    423322  423447  .       +       .       ID=exon:mk4.000002.13.3;Parent=transcript:mk4.000002.13
v31.000002      WormBase_imported       exon    423496  423636  .       +       .       ID=exon:mk4.000002.13.4;Parent=transcript:mk4.000002.13
v31.000002      WormBase_imported       five_prime_UTR  419767  419768  .       +       .       Parent=transcript:mk4.000002.13
v31.000002      WormBase_imported       CDS     419769  420048  .       +       1       ID=cds:mk4.000002.13;Parent=transcript:mk4.000002.13
v31.000002      WormBase_imported       CDS     420095  420198  .       +       0       ID=cds:mk4.000002.13;Parent=transcript:mk4.000002.13
v31.000002      WormBase_imported       CDS     423322  423447  .       +       0       ID=cds:mk4.000002.13;Parent=transcript:mk4.000002.13
v31.000002      WormBase_imported       CDS     423496  423636  .       +       0       ID=cds:mk4.000002.13;Parent=transcript:mk4.000002.13
"""
import re
planaria_example_gff = re.sub(r'[^\S\n]+', r'\t', planaria_example_gff)

# print(planaria_example_gff)

In [5]:
planaria = planaria_example_gff.splitlines()
planaria[0].split('\t')
gff_planaria = parse(planaria)
gff_planaria.specie = 'Wololo'

detected format gff3


In [6]:
for tranx in gff_planaria:
    annotation = GenomicAnnotation(
        starts=gff_planaria[tranx].exon_starts, ends=gff_planaria[tranx].exon_ends,
        strand=gff_planaria[tranx].strand, orientation=gff_planaria.orientation,
        cds_starts=gff_planaria[tranx].CDS_starts, cds_ends=gff_planaria[tranx].CDS_ends,
        chrom=gff_planaria[tranx].chrom, transcript_id=gff_planaria[tranx].transcript_id,
        gene_id=gff_planaria[tranx].gene_id
    )

In [7]:
annotation.exon_contrib_to_orf

array([ 0.43010753,  0.15975423,  0.19354838,  0.21658987], dtype=float32)

In [8]:
annotation.exons

array([282, 104, 126, 141])

In [9]:
annotation.exon_contrib_to_orf * annotation.orf_size

array([ 280.        ,  104.00000763,  126.        ,  141.        ], dtype=float32)

In [10]:
print(annotation.format('bed6'))

v31.000002	419766	420048	mk4.000002.13	1000	+
v31.000002	420094	420198	mk4.000002.13	1000	+
v31.000002	423321	423447	mk4.000002.13	1000	+
v31.000002	423495	423636	mk4.000002.13	1000	+



In [11]:
import pybedtools

In [12]:
?pybedtools.BedTool

In [13]:
a = pybedtools.BedTool(annotation.format('bed6'), from_string=True)

In [14]:
print(a.bed6())

v31.000002	419766	420048	mk4.000002.13	1000	+
v31.000002	420094	420198	mk4.000002.13	1000	+
v31.000002	423321	423447	mk4.000002.13	1000	+
v31.000002	423495	423636	mk4.000002.13	1000	+



In [15]:
sum(annotation.exons) + sum(annotation.introns) == annotation.end - annotation.start

True

In [16]:
# help(pybedtools.BedTool.merge)

In [17]:
print(a.intersect(a))

v31.000002	419766	420048	mk4.000002.13	1000	+
v31.000002	420094	420198	mk4.000002.13	1000	+
v31.000002	423321	423447	mk4.000002.13	1000	+
v31.000002	423495	423636	mk4.000002.13	1000	+



In [18]:
print(a.intersect(a).merge())

v31.000002	419766	420048
v31.000002	420094	420198
v31.000002	423321	423447
v31.000002	423495	423636



In [19]:
print(a.intersect(a).merge(d=50))

v31.000002	419766	420198
v31.000002	423321	423636



In [20]:
annotation.introns

array([  46, 3123,   48])

In [21]:
print(a.intersect(a).merge(d=50, c='4,5,6', o='distinct'))

v31.000002	419766	420198	mk4.000002.13	1000	+
v31.000002	423321	423636	mk4.000002.13	1000	+



In [34]:
def bed6_to_GeneAnnot(bed6):
     
    lines = bed6.splitlines()
    gannot = 0
    for line in lines:
        chrom, start, end, tranx_id, score, strand = line.split('\t')
        if gannot == 0:
            gannot = GenomicAnnotation(start, end, strand, 
                                       chrom=chrom, transcript_id=tranx_id, starts_offset=0)
        else:
            gannot.starts  = np.hstack([gannot.starts, int(start)])
            gannot.ends  = np.hstack([gannot.ends, int(end)])


    return gannot

def bed12_to_GeneAnnot(bed12):
    """
    01) chrom - The name of the chromosome (e.g. chr3, chrY, chr2_random) or scaffold (e.g. scaffold10671).
    02) chromStart - The starting position of the feature in the chromosome or scaffold. The first base in a chromosome is numbered 0.
    03) chromEnd - The ending position of the feature in the chromosome or scaffold.
    04) name - Defines the name of the BED line. This label is displayed to the left of the BED line in the Genome Browser window when the track is open to full display mode or directly to the left of the item in pack mode.
    05) score - A score between 0 and 1000.
    06) strand - Defines the strand - either '+' or '-'.
    07) thickStart - The starting position at which the feature is drawn thickly (for example, the start codon in gene displays). When there is no thick part, thickStart and thickEnd are usually set to the chromStart position.
    08) thickEnd - The ending position at which the feature is drawn thickly (for example, the stop codon in gene displays).
    09) itemRgb - An RGB value of the form R,G,B (e.g. 255,0,0). If the track line itemRgb attribute is set to "On", this RBG value will determine the display color of the data contained in this BED line. NOTE: It is recommended that a simple color scheme (eight colors or less) be used with this attribute to avoid overwhelming the color resources of the Genome Browser and your Internet browser.
    10) blockCount - The number of blocks (exons) in the BED line.
    11) blockSizes - A comma-separated list of the block sizes. The number of items in this list should correspond to blockCount.
    12) blockStarts - A comma-separated list of block starts. All of the blockStart positions should be calculated relative to chromStart. The number of items in this list should correspond to blockCount. 
    """
    
    bed12 = bed12.strip()
    bed_field = bed12.split('\t')
    starts = np.fromstring(bed_field[11])
    ends = np.fromstring(bed_field[10]) + starts
    name = bed_field[3]
    chrom = bed_field[0]
    strand = bed_field[5]
    
    return GenomicAnnotation(starts, ends, strand, chrom=chrom, transcript_id=name)
    
def merge_small_gap(annot, gap=20):
    from pybedtools import BedTool
    bed = BedTool(annot.format('bed6'), from_string=True)
    bed_merged = bed.intersect(bed).merge(d=gap, c='4,5,6', o='distinct')
    
    return bed6_to_GeneAnnot(str(bed_merged))            

In [35]:
bed6_to_GeneAnnot(annotation.format('bed6')).exons == annotation.exons

array([ True,  True,  True,  True], dtype=bool)

In [25]:
merge_small_gap(annotation).exons

array([282, 104, 126, 141])

In [26]:
merge_small_gap(annotation, gap=50).exons

array([432, 315])

In [43]:
annotation.introns < 50

array([ True, False,  True], dtype=bool)

In [48]:
print(str(a) + str(a))

v31.000002	419766	420048	mk4.000002.13	1000	+
v31.000002	420094	420198	mk4.000002.13	1000	+
v31.000002	423321	423447	mk4.000002.13	1000	+
v31.000002	423495	423636	mk4.000002.13	1000	+
v31.000002	419766	420048	mk4.000002.13	1000	+
v31.000002	420094	420198	mk4.000002.13	1000	+
v31.000002	423321	423447	mk4.000002.13	1000	+
v31.000002	423495	423636	mk4.000002.13	1000	+

